In [45]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Conv1D, Dropout, Add, Dense, Input, BatchNormalization, Activation, Lambda
from tensorflow.keras.callbacks import EarlyStopping

# Carregar os dados
df = pd.read_csv("/content/inflacao.csv")
df['data'] = pd.to_datetime(df['referencia'])
df.set_index('data', inplace=True)

# Selecionar a coluna de interesse
data = df['ipca_variacao'].values.reshape(-1, 1)

# Normalizar os dados
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# Função para criar conjuntos de dados
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

# Definir o número de passos de tempo
time_step = 12


# Selecionar a coluna de interesse
data = df['ipca_variacao'].values.reshape(-1, 1)

# Normalizar os dados
scaler = MinMaxScaler(feature_range=(0, 1))
data_scaled = scaler.fit_transform(data)

# Função para criar conjuntos de dados
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

# Definir o número de passos de tempo
time_step = 12


In [46]:
# Criar modelos
def create_tanh_rnn_model(input_shape):
    model = Sequential()
    model.add(SimpleRNN(50, activation='tanh', input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, activation='tanh', input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def create_gru_model(input_shape):
    model = Sequential()
    model.add(GRU(50, activation='tanh', input_shape=input_shape))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def create_tcn_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv1D(filters=64, kernel_size=3, padding='causal', activation='relu', input_shape=input_shape)(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.2)(x)
    x = Conv1D(filters=64, kernel_size=3, padding='causal', activation='relu', input_shape=input_shape)(x)
    x = BatchNormalization()(x)

    # Conexão residual
    if inputs.shape[-1] == x.shape[-1]:
        skip_connection = inputs
    else:
        skip_connection = Conv1D(filters=64, kernel_size=1, padding='same')(inputs)

    x = Add()([x, skip_connection])
    x = Activation('relu')(x)

    # Pooling
    x = Lambda(lambda x: x[:, -1, :])(x)

    x = Dense(1)(x)
    model = Model(inputs=inputs, outputs=x)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


In [47]:
# Validação cruzada de séries temporais
tscv = TimeSeriesSplit(n_splits=5)
rmse_results = {}

for model_name, create_model in [('TANH RNN', create_tanh_rnn_model),
                                  ('LSTM', create_lstm_model),
                                  ('GRU', create_gru_model),
                                  ('TCN', create_tcn_model)]:
    rmse_fold = []

    for train_index, test_index in tscv.split(data_scaled):
        X_train, y_train = create_dataset(data_scaled[train_index], time_step)
        X_test, y_test = create_dataset(data_scaled[test_index], time_step)

        # Reshape para [amostras, passos de tempo, recursos]
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

        # Criar e treinar o modelo
        model = create_model((X_train.shape[1], 1))
        early_stopping = EarlyStopping(monitor='loss', patience=5)
        model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=0, callbacks=[early_stopping])

        # Previsões
        y_pred = model.predict(X_test)

        # Remover a dimensão extra de y_pred
        y_pred = y_pred.reshape(-1)

        # Calcular RMSE
        if y_test.shape[0] == y_pred.shape[0]:
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            rmse_fold.append(rmse)
        else:
            print("Tamanhos diferentes: não calculando RMSE.")
            break;

    rmse_results[model_name] = np.mean(rmse_fold) if rmse_fold else np.nan


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step


In [48]:
# Mostrar resultados
for model, rmse in rmse_results.items():
    print(f"{model} RMSE: {rmse:.4f}")

TANH RNN RMSE: 0.0369
LSTM RMSE: 0.0556
GRU RMSE: 0.0493
TCN RMSE: 0.0838
